In [ ]:

# 사전설치 : pip install kss textblob torch
# 허깅페이스 로그인 접속 : pip install huggingface_hub, 터미널에서 huggingface-cli login, settings> Access Token에서 토큰 생성 후 액세스 토큰 입력
import os
import gradio as gr
import requests
import re
import json
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import kss  # kss는 "Korean Sentence Splitter"의 약자로, 한국어 문장 분리 도구
from textblob import TextBlob
import numpy as np
import torch
import warnings
warnings.filterwarnings('ignore')   # 파이썬의 warnings 모듈을 사용하여 발생하는 경고 메시지를 무시

In [ ]:
# Gemini API 키 설정
os.environ["GEMINI_API_KEY"] = "insert your api key"  # 발급받은 Gemini API 키

In [ ]:
class NewsAnalyzer:
    def __init__(self):
        print("모델 초기화 중...")

        # 감성 분석 모델 초기화
        print("감성 분석 모델 로딩...")
        self.sentiment_analyzer = pipeline(
            "sentiment-analysis",
            model="snunlp/KR-FinBert-SC",
            tokenizer="snunlp/KR-FinBert-SC"
        )

        # 가짜 뉴스 분류 모델 초기화
        print("가짜 뉴스 분류 모델 로딩...")
        self.init_fake_news_classifier()

        print("모델 초기화 완료!")

    def init_fake_news_classifier(self):
        """가짜 뉴스 분류 모델 초기화"""
        try:
            # 'beomi/kcbert-base' 모델 사용 - 한국어에 더 특화된 모델
            model_name = "beomi/kcbert-base"
            self.fake_news_classifier = pipeline(
                "text-classification",
                model=model_name,
                tokenizer=model_name
            )

            def classify_long_text(text):
                try:
                    # 한국어 문장 분리
                    sentences = kss.split_sentences(text)  # 텍스트를 문장 단위로 분리
                    chunks = []
                    current_chunk = ""

                    # 청크 생성
                    for sentence in sentences:
                        if len(current_chunk) + len(sentence) < 1000:  # 1000자 미만의 청크 생성
                            current_chunk += sentence + " "
                        else:
                            if current_chunk:
                                chunks.append(current_chunk.strip())
                            current_chunk = sentence + " "
                    if current_chunk:
                        chunks.append(current_chunk.strip())

                    if not chunks:
                        chunks = [text]

                    # 각 청크에 대해 분류 수행
                    results = []
                    for chunk in chunks:
                        try:
                            result = self.fake_news_classifier(chunk[:512])  # 청크의 처음 512자만 사용하도록 제한

                            # 신뢰도 점수 계산 - 문장의 특성에 따라 가중치 부여
                            score = result[0]['score']

                            # 가짜뉴스 특성 체크
                            fake_news_indicators = self.check_fake_news_indicators(chunk)  # 청크에서 가짜 뉴스 관련 특징을 확인

                            # 최종 점수 조정
                            adjusted_score = self.adjust_score(score, fake_news_indicators)  # 기본 점수를 가짜 뉴스 특징에 따라 조정
                            results.append(adjusted_score)

                        except Exception as e:
                            print(f"청크 처리 중 오류: {str(e)}")
                            continue

                    # 결과 평균 계산 및 레이블 결정
                    if results:
                        avg_score = np.mean(results)  # 청크들의 점수 평균을 계산
                        # 점수가 0.6 이상일 때 가짜뉴스로 판정 (임계값 조정)
                        final_label = 'FAKE' if avg_score > 0.6 else 'REAL'
                        return [{"label": final_label, "score": avg_score}]
                    else:
                        return [{"label": "UNKNOWN", "score": 0.5}]

                except Exception as e:
                    print(f"텍스트 분류 중 오류: {str(e)}")
                    return [{"label": "UNKNOWN", "score": 0.5}]

            self.classify_text = classify_long_text

        except Exception as e:
            print(f"가짜 뉴스 분류 모델 로딩 실패: {str(e)}")
            self.classify_text = lambda x: [{"label": "UNKNOWN", "score": 0.5}]

    def check_fake_news_indicators(self, text):
        """가짜뉴스 특성 체크"""
        indicators = {
            'excessive_punctuation': len(re.findall(r'[!?]{2,}', text)) > 0,
            'all_caps_words': len(re.findall(r'\b[A-Z가-힣]{3,}\b', text)) > 0,
            'sensational_words': any(word in text.lower() for word in [
                '충격', '경악', '발칵', '화들짝', '헉', '대박', '경악',
                '충격적', '전격', '초특급', '극비', '특종', '단독'
            ]),
            'unverified_sources': any(phrase in text for phrase in [
                '카더라', '라고 한다', '했다고 한다', '라는 소문',
                '알려졌다', '전해졌다', '...', '소식통'
            ])
        }
        return indicators

    def adjust_score(self, base_score, indicators):
        """지표에 따른 점수 조정"""
        score = base_score

        # 각 지표별 가중치 적용
        if indicators['excessive_punctuation']:  # 과도한 문장 부호 사용
            score += 0.1
        if indicators['all_caps_words']:  # 모든 글자가 대문자인 단어 사용
            score += 0.1
        if indicators['sensational_words']:  # 선정적인 단어 사용
            score += 0.15
        if indicators['unverified_sources']:  # 확인되지 않은 출처 인용
            score += 0.15

        # 점수 범위 제한 (0~1)
        return min(max(score, 0), 1)

    def analyze_with_gemini(self, article):
        """Gemini API를 사용한 분석"""
        if not os.environ.get("GEMINI_API_KEY") or os.environ["GEMINI_API_KEY"] == "your-gemini-api-key-here":
            return "Gemini API 키가 설정되지 않았습니다."

        try:
            url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"
            headers = {
                'Content-Type': 'application/json',
                'x-goog-api-key': os.environ['GEMINI_API_KEY']
            }

            max_chunk_length = 4000
            chunks = [article[i:i + max_chunk_length] for i in range(0, len(article), max_chunk_length)]

            analyses = []
            for chunk in chunks[:3]:
                prompt = (
                    "다음 뉴스 기사의 진위 여부를 분석해주세요. 다음 항목별로 평가해주세요:\n"
                    "1. 기사의 객관성\n"
                    "2. 사실 확인 가능한 정보의 존재 여부\n"
                    "3. 감정적 표현이나 과장된 표현의 사용\n"
                    "4. 출처와 인용의 명확성\n\n"
                    f"기사 내용:\n{chunk}"
                )

                data = {"contents": [{"parts": [{"text": prompt}]}]}

                response = requests.post(url, headers=headers, json=data)
                response.raise_for_status()
                result = response.json()
                analyses.append(result['candidates'][0]['content']['parts'][0]['text'])

            return "\n\n".join(analyses)

        except Exception as e:
            return f"Gemini 분석 중 오류 발생: {str(e)}"

    def analyze_sentiment_details(self, article):
        """세부적인 감성 분석"""
        try:
            sentences = kss.split_sentences(article)
            sentiment_scores = []

            for sentence in sentences[:50]:
                try:
                    sentiment_result = self.sentiment_analyzer(sentence[:512])[0]
                    score = (float(sentiment_result['score']) - 0.5) * 2
                    sentiment_scores.append(score)
                except Exception as e:
                    print(f"문장 감성 분석 중 오류: {str(e)}")
                    continue

            if sentiment_scores:
                return {
                    "평균_감성점수": np.mean(sentiment_scores),
                    "감성_변동성": np.std(sentiment_scores),
                    "극단적_문장_비율": len([s for s in sentiment_scores if abs(s) > 0.5]) / len(sentiment_scores)
                }
            else:
                return {
                    "평균_감성점수": 0,
                    "감성_변동성": 0,
                    "극단적_문장_비율": 0
                }
        except Exception as e:
            print(f"감성 분석 중 오류 발생: {str(e)}")
            return {
                "평균_감성점수": 0,
                "감성_변동성": 0,
                "극단적_문장_비율": 0
            }

    def analyze_news(self, article):
        """통합 뉴스 분석"""
        if not article or len(article.strip()) == 0:
            return "분석할 텍스트를 입력해주세요."

        try:
            # 1. Gemini 분석
            gemini_analysis = self.analyze_with_gemini(article)

            # 2. 가짜 뉴스 분류기 분석
            fake_news_result = self.classify_text(article)

            # 3. 감성 분석
            sentiment_details = self.analyze_sentiment_details(article)

            # 4. 신뢰도 계산 (가짜뉴스 점수를 반전)
            credibility_score = (1 - float(fake_news_result[0]['score'])) * 100

            # 판정 기준 변경: 점수가 40% 미만이면 가짜뉴스
            is_fake = credibility_score < 40

            # 결과 텍스트 생성
            result = f"""
=== 뉴스 진위 분석 결과 ===

[최종 판정]
{'가짜 뉴스' if is_fake else '진짜 뉴스'} (신뢰도: {round(credibility_score, 2)}%)

[상세 분석]
1. AI 모델 분석:
   - 판정: {'FAKE' if is_fake else 'REAL'}
   - 신뢰도: {round(credibility_score, 2)}%

2. 감성 분석:
   - 감정 강도: {round(abs(sentiment_details['평균_감성점수']) * 100, 2)}%
   - 극단적 표현 비율: {round(sentiment_details['극단적_문장_비율'] * 100, 2)}%

3. Gemini 분석:
{gemini_analysis}

※ 주의: 이 분석은 참고용이며, 최종적인 판단은 전문가의 검증이 필요합니다.
"""
            return result

        except Exception as e:
            return f"""
=== 오류 발생 ===
분석 중 문제가 발생했습니다: {str(e)}
시스템 관리자에게 문의하세요.
"""

In [ ]:
def create_interface():
    analyzer = NewsAnalyzer()

    iface = gr.Interface(
        fn=analyzer.analyze_news,
        inputs=gr.Textbox(lines=10, placeholder="뉴스 기사 내용을 입력하세요..."),
        outputs=gr.Textbox(lines=20),
        title="가짜 뉴스 판별 시스템",
        description="AI 모델을 활용한 뉴스 진위 판별 시스템"
    )
    return iface

In [ ]:
if __name__ == "__main__":
    analyzer = NewsAnalyzer()
    iface = create_interface()
    iface.launch()

모델 초기화 중...
감성 분석 모델 로딩...


Device set to use cpu


가짜 뉴스 분류 모델 로딩...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


모델 초기화 완료!
모델 초기화 중...
감성 분석 모델 로딩...


Device set to use cpu


가짜 뉴스 분류 모델 로딩...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


모델 초기화 완료!
* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
iface.close()

Closing server running on port: 7860
